Neste notebook é criado e configurado o Webhook que receberá a requisição manual e enviará as informações para o servidor Kafka.

In [0]:
pip install flask fastapi uvicorn pyngrok kafka-python

Python interpreter will be restarted.
  Created wheel for pyngrok: filename=pyngrok-7.0.0-py3-none-any.whl size=21144 sha256=9dd26f2a540848b63e1d41e0556507574b498088fcd0fe559596b8130d871de7
  Stored in directory: /root/.cache/pip/wheels/a1/27/d7/83363a0fd832c7e4be4a73ae8805f9b5410b40570ae8be7d67
Successfully built pyngrok
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.1.1
    Not uninstalling typing-extensions at /databricks/python3/lib/python3.9/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-8b153dcc-568f-4ebc-8bf7-7ecf9707d32e
    Can't uninstall 'typing-extensions'. No files were found to uninstall.
  Attempting uninstall: MarkupSafe
    Found existing installation: MarkupSafe 2.0.1
    Not uninstalling markupsafe at /databricks/python3/lib/python3.9/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-8b153dcc-568f-4ebc-8bf7-7ecf9707d32e
    Can't uninstall 'MarkupSafe'. No files

In [0]:
from kafka import KafkaProducer
from pyngrok import ngrok
from fastapi import FastAPI, Request
import uvicorn,json

app = FastAPI()

@app.post('/webhook')
async def webhook(request: Request):
    data = await request.json()
    producer = KafkaProducer(bootstrap_servers=['localhost:9092'])
    json_data = data
    message = json.dumps(json_data).encode('utf-8')
    topic = 'responseManualRequest'
    producer.send(topic, message)

    return f'Manual Request Received! Kafka topic: {topic}]'

port = 8000
ngrok_tunnel = ngrok.connect(port)

print('-------------------------------------------------')
print('Public URL:', ngrok_tunnel.public_url)
print('-------------------------------------------------')
config = uvicorn.Config(app)
server = uvicorn.Server(config)
await server.serve()

t=2023-10-04T23:16:13+0000 lvl=warn msg="ngrok config file found at legacy location, move to XDG location" xdg_path=/root/.config/ngrok/ngrok.yml legacy_path=/root/.ngrok2/ngrok.yml
INFO:     Started server process [18138]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


-------------------------------------------------
Public URL: https://462b-35-90-241-14.ngrok.io
-------------------------------------------------
INFO:     35.90.241.14:0 - "POST /webhook HTTP/1.1" 200 OK
